In [4]:
!pip install tflearn -q


[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nanik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Scipy not supported!


In [15]:
with open('../jsondata/intents.json') as f:
    data = json.load(f)

In [27]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\nanik\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [28]:
from nltk.tokenize import word_tokenize
print(word_tokenize("Hello, how are you?"))


['Hello', ',', 'how', 'are', 'you', '?']


In [29]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
  for pattern in intent["patterns"]:
    wrds = nltk.word_tokenize(pattern)
    words.extend(wrds)
    docs_x.append(wrds)
    docs_y.append(intent["tag"])


In [30]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
  for pattern in intent["patterns"]:
    wrds = nltk.word_tokenize(pattern)
    words.extend(wrds)
    docs_x.append(wrds)
    docs_y.append(intent["tag"])


  if intent["tag"] not in labels:
    labels.append(intent["tag"])  

words = [stemmer.stem(w.lower()) for w in words if  w not in "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x,doc in enumerate(docs_x):
  bag = []
  wrds = [stemmer.stem(w) for w in doc]

  for w in words:
    if w in wrds:
      bag.append(1)
    else:
      bag.append(0)

  output_row = out_empty[:]
  output_row[labels.index(docs_y[x])] = 1

  training.append(bag)
  output.append(output_row)

training = np.array(training)
output = np.array(output)

tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation = "softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



In [31]:
model.fit(training, output, n_epoch = 1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 23999  | total loss: 0.00037 | time: 0.028s
| Adam | epoch: 1000 | loss: 0.00037 - acc: 1.0000 -- iter: 184/188
Training Step: 24000  | total loss: 0.00036 | time: 0.028s
| Adam | epoch: 1000 | loss: 0.00036 - acc: 1.0000 -- iter: 188/188
--
INFO:tensorflow:C:\Users\nanik\Desktop\Karthik\kaggale\notebooks\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [32]:
def bag_of_words(s, words):
  bag = [0 for _ in range(len(words))]

  s_words = nltk.word_tokenize(s)
  s_words = [stemmer.stem(word.lower()) for word in s_words]

  for se in s_words:
    for i, w in enumerate(words):
      if w == se:
        bag[i] = 1

  return np.array(bag)

def chat():
  print("\033[1;31m Start talking with the bot!, Type quit when you want to quit")
  while True:
    inp = input("\033[1;31m You: ")
    if inp.lower() == "quit":
      break

    results = model.predict([bag_of_words(inp, words)])
    results_index = np.argmax(results)
    tag = labels[results_index]

    for tg in data["intents"]:
      
      if tg['tag'] == tag:
        
        responses = tg['responses']

        response_list = nltk.sent_tokenize(str(responses[0]))
        colors = [31, 32, 33, 34, 35]

        for i in range(len(response_list)):
            color_index = i % len(colors)
            print(f'\033[1;{colors[color_index]}m {response_list[i]:<12s}\n')


In [ ]:
chat()

 Start talking with the bot!, Type quit when you want to quit


 You:  hii


 Begin with washed hands.Gently clean the area with cool to lukewarm water and mild soap.

 Remove dirt or other particles from the wound using sterilized tweezers.For a mild scrape thatâ€™s not bleeding, leave the wound uncovered.If the wound is bleeding, use a clean cloth or bandage, and apply gentle pressure to the area to stop any bleeding.Cover a wound that bled with a thin layer of topical antibiotic ointment, like Bacitracin, or a sterile moisture barrier ointment, like Aquaphor.

 Cover it with a clean bandage or gauze.

 Gently clean the wound and change the ointment and bandage once per day.Watch the area for signs of infection, like pain or redness and swelling.

 See your doctor if you suspect infection.



 You:  feeling headache


 Give ibuprofen (Advil, Motrin), aspirin, or acetaminophen (Tylenol) for pain.

 Avoid ibuprofen and other NSAIDs if the person has heart failure or kidney failure.

 Do not give aspirin to a child under age 18.



...

 Give ibuprofen (Advil, Motrin), aspirin, or acetaminophen (Tylenol) for pain.

 Avoid ibuprofen and other NSAIDs if the person has heart failure or kidney failure.

 Do not give aspirin to a child under age 18.



 You:  need tablet for cold


 1)Keeping hydrated is absolutely vital to help 'flush' out the cold, as well as to break down congestion and keep your throat lubricated.

 2)Vitamin C is extremely helpful when fighting infection, so at the first sign of a cold be sure to increase your intake by eating plenty of berries, citrus fruits, papayas, broccoli and red peppers which will help keep you protected.

 3)When it comes to combating a cold,Vitamin D is essential in helping to regulate immune response.



 You:  need tablet for fever


 To treat a fever at home: 1)Drink plenty of fluids to stay hydrated.

 2)Dress in lightweight clothing.

 3)Use a light blanket if you feel chilled, until the chills end.

 4)Take acetaminophen (Tylenol, others) or ibuprofen (Advil, Motrin IB, others).

 5) Get medical help if the fever lasts more than five days in a row.

